In [1]:
import pandas as pd
import numpy as np

In [2]:
scenario = 'real'

In [3]:
top_down = range(1024)
bottom_up = range(1024, 2048)

In [4]:
data_train = pd.read_csv('./data/{}_q_types_features_train.csv'.format(scenario), header=None)
#data_train['_scen'] = 'train'
data_val = pd.read_csv('./data/{}_q_types_features_val.csv'.format(scenario), header=None)
#data_val['_scen'] = 'val'
data = pd.concat([data_train, data_val])
del data_train
del data_val

In [5]:
data[2048] = data[2048].astype(int)
data.sort_values(by=2048, inplace=True)

In [6]:
data.shape

(658111, 2050)

In [7]:
details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
details = pd.concat([details_train, details_val])
details[2] = details[2].astype(int)
details.sort_values(by=2, inplace=True)

predicted = details[0].values
target = details[1].values

data['my_predicted'] = predicted
data['my_target'] = target

In [8]:
(data[2048] != details[2].values).astype(int).sum()

0

In [9]:
details_train = pd.read_csv('./data/predictions_details_base_{}_train'.format(scenario), header=None)
details_val = pd.read_csv('./data/predictions_details_base_{}_val'.format(scenario), header=None)
details = pd.concat([details_train, details_val])
details[2] = details[2].astype(int)
details.sort_values(by=2, inplace=True)

predicted = details[0].values
target = details[1].values
data['base_predicted'] = predicted
data['base_target'] = target

In [10]:
(data[2048] != details[2].values).astype(int).sum()

0

In [11]:
#labels_train = pd.read_csv('./data/predictions_{}_train'.format(scenario), header=None).values[:,0]
#labels_val = pd.read_csv('./data/predictions_{}_val'.format(scenario), header=None).values[:,0]
#labels = labels_train.tolist() + labels_val.tolist()
#data['my_acc'] = labels
#assert labels_train.shape[0] + labels_val.shape[0] == len(labels)

In [12]:
#data.to_csv('./concatenated_data/{}.csv'.format(scenario), index=False)

In [13]:
data[:3]

,0,1,2,3,4,5,6,7,8,9,...,2044,2045,2046,2047,2048,2049,my_predicted,my_target,base_predicted,base_target
0,3.621112,0.0,0.362051,0.0,0.0,0.0,0.0,0.0,0.0,3.502063,...,0.723560,1.716353,0.000000,2.856147,9000,how many,841,841,2195,841
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,2.273109,0.000000,7.530718,0.654848,9001,what color are the,1204,2542,759,2542
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,14.255922,0.000000,1.289435,0.000000,9002,what is the,2529,2529,2529,2529


In [14]:
data['sum_top_down'] = data[top_down].sum(1)
data['sum_bottom_up'] = data[bottom_up].sum(1)

In [15]:
data[:3]

,0,1,2,3,4,5,6,7,8,9,...,2046,2047,2048,2049,my_predicted,my_target,base_predicted,base_target,sum_top_down,sum_bottom_up
0,3.621112,0.0,0.362051,0.0,0.0,0.0,0.0,0.0,0.0,3.502063,...,0.000000,2.856147,9000,how many,841,841,2195,841,515.023036,555.346463
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,7.530718,0.654848,9001,what color are the,1204,2542,759,2542,735.734441,733.956401
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,1.289435,0.000000,9002,what is the,2529,2529,2529,2529,946.130073,1108.889205


In [16]:
data[data.columns.tolist()[-8:]].to_csv('./concatenated_data/{}_prediction.csv'.format(scenario), index=False)

In [17]:
(data[2049]>0).sum()

658111

In [18]:
data.shape

(658111, 2056)

In [19]:
# column 2048 -> query type
_sum = data.groupby(2049).sum().round(1)
count = data.groupby(2049).count()

activations_sum = pd.concat([pd.DataFrame(_sum[top_down].sum(1), columns=['sum_top_down']), 
                           pd.DataFrame(_sum[bottom_up].sum(1), columns=['sum_bottom_up'])], 1)
_sum = (activations_sum.sum_top_down + activations_sum.sum_bottom_up).values
activations_sum = activations_sum/np.array([_sum, _sum]).T

activations_count = pd.concat([pd.DataFrame(count[top_down].sum(1), columns=['count_top_down']), 
                             pd.DataFrame(count[bottom_up].sum(1), columns=['count_bottom_up'])], 1)
_sum = (activations_count.count_top_down + activations_count.count_bottom_up).values
activations_count = activations_count/np.array([_sum, _sum]).T

summary = pd.concat([activations_sum, activations_count], 1)

In [20]:
summary

,sum_top_down,sum_bottom_up,count_top_down,count_bottom_up
2049,,,,
4 lettered,0.440907,0.559093,0.5,0.5
a pull,0.475547,0.524453,0.5,0.5
a red,0.514597,0.485403,0.5,0.5
a type,0.512143,0.487857,0.5,0.5
a white,0.505314,0.494686,0.5,0.5
according to,0.502500,0.497500,0.5,0.5
adult or,0.482540,0.517460,0.5,0.5
am i,0.499086,0.500914,0.5,0.5
android or,0.535405,0.464595,0.5,0.5


In [21]:
#p = 1.5 #90%

mean_dif = (summary.sum_top_down - summary.sum_bottom_up).mean()
std_dif = (summary.sum_top_down - summary.sum_bottom_up).std()
data_dif = (summary.sum_top_down - summary.sum_bottom_up)
#data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
#                         (data_dif > mean_dif + p * std_dif).astype(int) > 0 )

significantly_different = summary.copy()#.loc[data_diff[data_diff[0]].index]
significantly_different['diff'] = data_dif.round(2)
abstract_rep_count = pd.DataFrame(data[2049].value_counts())
abstract_representativity = np.round(abstract_rep_count/data.shape[0] *100, 4)
print(abstract_representativity.shape)
significantly_different['_count'] = abstract_rep_count.loc[significantly_different.index]
#significantly_different['represents %'] = abstract_representativity.loc[significantly_different.index]
significantly_different['over_median'] = abstract_representativity.loc[significantly_different.index] > float(abstract_representativity.median())
print(float(abstract_representativity.median()))
significantly_different['winner'] = np.array(['top_down',
                                              'bottom_up'])[(significantly_different.sum_bottom_up 
                                                            > significantly_different.sum_top_down).astype(int)]

#np.round(float(data_dif[data_diff[0]].shape[0]) / data_diff.shape[0] * 100, 2)

(632, 1)
0.0007


In [22]:
significantly_different.shape

(632, 8)

In [23]:
significantly_different.to_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario), index=True)

In [24]:
significantly_different#[significantly_different._count>1].sort_values(by='winner')

,sum_top_down,sum_bottom_up,count_top_down,count_bottom_up,diff,_count,over_median,winner
2049,,,,,,,,
4 lettered,0.440907,0.559093,0.5,0.5,-0.12,1,False,bottom_up
a pull,0.475547,0.524453,0.5,0.5,-0.05,1,False,bottom_up
a red,0.514597,0.485403,0.5,0.5,0.03,2,False,top_down
a type,0.512143,0.487857,0.5,0.5,0.02,2,False,top_down
a white,0.505314,0.494686,0.5,0.5,0.01,1,False,top_down
according to,0.502500,0.497500,0.5,0.5,0.01,2,False,top_down
adult or,0.482540,0.517460,0.5,0.5,-0.03,2,False,bottom_up
am i,0.499086,0.500914,0.5,0.5,-0.00,2,False,bottom_up
android or,0.535405,0.464595,0.5,0.5,0.07,2,False,top_down
